# Implementación del Desafío 4 (PNL1)

**Alumno:**
- Adassus, Luciano (CEIA 17Co2024)

**Objetivo:**

El objecto es utilizar datos disponibles del challenge ConvAI2 (Conversational Intelligence Challenge 2) de conversaciones en inglés. Se construirá un BOT, como primer versión, para responder a preguntas del usuario (QA).\ [LINK](http://convai.io/data/)

In [186]:
import pandas as pd
import requests
import json
import numpy as np
import gensim.downloader as api
import gensim
import nltk
import re
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Dot, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("NumPy:", np.__version__)        # Deberia ser 1.26.4
print("TensorFlow:", tf.__version__)   # Deberia ser 2.16.1
print("Gensim:", gensim.__version__)   # Deberia ser 4.3.3

NumPy: 1.26.4
TensorFlow: 2.16.2
Gensim: 4.3.3


### 🚩 1. Datos

In [187]:
# Descargar dataset ConvAI2 (train)
url = "http://convai.io/data/summer_wild_evaluation_dialogs.json"
response = requests.get(url)
convai2_json = response.json()

# Extraer pares pregunta-respuesta claros
questions = []
answers = []

# ConvAI2 tiene estructura anidada
for dialog in convai2_json:
    utterances = dialog['dialog']
    # Guardar pares consecutivos como pregunta y respuesta
    for i in range(len(utterances)-1):
        questions.append(utterances[i]['text'])
        answers.append(utterances[i+1]['text'])

# Crear DataFrame
df = pd.DataFrame({'question': questions, 'answer': answers})
# Usar solo las primeras 15.000 filas (en vez de 43.739) por temas de memoria RAM
df = df.iloc[:15000].copy()
df.head()

,question,answer
0,I love iphone! i just bought new iphone!,"Thats good for you, i'm not very into new tech"
1,"Thats good for you, i'm not very into new tech",I am a college student and i am a college student
2,I am a college student and i am a college student,I am go to gym and live on donations
3,I am go to gym and live on donations,I am a vegan and i am in the midwest
4,I am a vegan and i am in the midwest,So vegan... i have dogs maybe i should told th...


### 🚩 2. Preprocesamiento

In [188]:
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.data.path.append('/usr/nltk_data')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lucianoadassus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lucianoadassus/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#### Limpieza:

In [189]:
# Diccionario de contracciones
CONTRACTIONS = {
    "i'm": "i am", "you're": "you are", "he's": "he is", "she's": "she is",
    "it's": "it is", "we're": "we are", "they're": "they are", "can't": "cannot",
    "won't": "will not", "don't": "do not", "didn't": "did not", "i've": "i have",
    "i'll": "i will", "you'll": "you will", "she'd": "she would", "should've": "should have",
    "there's": "there is", "we'd": "we would", "they'll": "they will", "wasn't": "was not"
}

# Expandir contracciones
def expand_contractions(text):
    pattern = re.compile(r'\b(' + '|'.join(CONTRACTIONS.keys()) + r')\b')
    return pattern.sub(lambda x: CONTRACTIONS[x.group()], text)

# Función de limpieza completa
def clean_text(text):
    text = text.lower()
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'http\S+', '', text)
    text = expand_contractions(text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'[\u263a-\U0001f645]', ' ', text)
    text = re.sub(r'[\u2600-\u26FF]', ' ', text)
    tokens = word_tokenize(text)
    return ' '.join(tokens)

# Aplicar limpieza
df['question_clean'] = df['question'].apply(clean_text)
df['answer_clean'] = df['answer'].apply(clean_text)

# ➕ Agregar tokens <sos> y <eos>
df['answer_clean'] = df['answer_clean'].apply(lambda x: '<sos> ' + x.strip() + ' <eos>')

# 🧯 Filtro y balanceo del corpus
df = df.dropna(subset=['question_clean', 'answer_clean'])
df = df[(df['question_clean'].str.strip() != '') & (df['answer_clean'].str.strip() != '')]
df = df[df['question_clean'] != df['answer_clean']]
df = df[df['answer_clean'].str.split().apply(len) > 3]
df = df.drop_duplicates(subset=['question_clean', 'answer_clean'])

# Vista previa
print(f"✅ Total de pares tras limpieza: {len(df)}")
display(df[['question_clean', 'answer_clean']].head())

/var/folders/wj/y24k5tss16n7pqyxn9b_g9x80000gp/T/ipykernel_2066/3403826205.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text
/var/folders/wj/y24k5tss16n7pqyxn9b_g9x80000gp/T/ipykernel_2066/3403826205.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


✅ Total de pares tras limpieza: 12648


,question_clean,answer_clean
0,i love iphone i just bought new iphone,<sos> thats good for you i am not very into ne...
1,thats good for you i am not very into new tech,<sos> i am a college student and i am a colleg...
2,i am a college student and i am a college student,<sos> i am go to gym and live on donations <eos>
3,i am go to gym and live on donations,<sos> i am a vegan and i am in the midwest <eos>
4,i am a vegan and i am in the midwest,<sos> so vegan i have dogs maybe i should told...


####  Generar tokenizers, secuencias y tensores de entrenamiento:

In [190]:
# Parámetros
MAX_VOCAB_SIZE = 6000
MAX_SEQ_LENGTH = 25

# Tokenizadores
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='', lower=True)
tokenizer_inputs.fit_on_texts(df['question_clean'])
input_sequences = tokenizer_inputs.texts_to_sequences(df['question_clean'])

tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='', lower=True)
tokenizer_outputs.fit_on_texts(df['answer_clean'])
output_sequences = tokenizer_outputs.texts_to_sequences(df['answer_clean'])

# Diccionarios
word2idx_inputs = tokenizer_inputs.word_index
word2idx_outputs = tokenizer_outputs.word_index
num_words_output = min(MAX_VOCAB_SIZE, len(word2idx_outputs) + 1)

# Longitudes máximas
max_input_len = min(MAX_SEQ_LENGTH, max(len(seq) for seq in input_sequences))
max_out_len = min(MAX_SEQ_LENGTH, max(len(seq) for seq in output_sequences))

# Padding
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
decoder_output_sequences = pad_sequences(output_sequences, maxlen=max_out_len, padding='post')

# Crear decoder targets desplazados
decoder_targets = np.zeros((len(decoder_output_sequences), max_out_len, num_words_output), dtype='float32')
for i, seq in enumerate(decoder_output_sequences):
    for t, word_idx in enumerate(seq):
        if t > 0:
            decoder_targets[i, t - 1, word_idx] = 1.0

# Prints informativos
print(f"✅ Total de pares pregunta-respuesta: {len(df)}")
print(f"📚 Vocabulario input: {min(len(word2idx_inputs), MAX_VOCAB_SIZE)}")
print(f"📚 Vocabulario output: {num_words_output}")
print(f"🧱 Longitud máxima input: {max_input_len}")
print(f"🧱 Longitud máxima output: {max_out_len}")
print(f"🎯 decoder_targets shape: {decoder_targets.shape}")

# Mostrar ejemplo
print("\n🔎 Ejemplo:")
print(f"Pregunta original: {df['question_clean'].iloc[0]}")
print(f"Secuencia tokenizada: {input_sequences[0]}")
print(f"Secuencia padded: {encoder_input_sequences[0]}")
print(f"Respuesta tokenizada: {output_sequences[0]}")
print(f"Respuesta padded: {decoder_output_sequences[0]}")

✅ Total de pares pregunta-respuesta: 12648
📚 Vocabulario input: 4046
📚 Vocabulario output: 4171
🧱 Longitud máxima input: 25
🧱 Longitud máxima output: 25
🎯 decoder_targets shape: (12648, 25, 4171)

🔎 Ejemplo:
Pregunta original: i love iphone i just bought new iphone
Secuencia tokenizada: [1, 20, 855, 1, 38, 739, 145, 855]
Secuencia padded: [  1  20 855   1  38 739 145 855   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
Respuesta tokenizada: [1, 53, 27, 26, 4, 3, 5, 9, 52, 229, 154, 2236, 2]
Respuesta padded: [   1   53   27   26    4    3    5    9   52  229  154 2236    2    0
    0    0    0    0    0    0    0    0    0    0    0]


### 🚩 3. Preparar los embeddings 

In [191]:
# Descargar FastText
print("⏬ Cargando FastText...")
fasttext = api.load('fasttext-wiki-news-subwords-300')

embedding_dim = 300
embedding_matrix = np.zeros((MAX_VOCAB_SIZE, embedding_dim))

# Contadores para estadísticas
found = 0
missing = []

print("🔄 Generando embedding matrix...")

for word, i in word2idx_inputs.items():
    if i < MAX_VOCAB_SIZE:
        if word in fasttext:
            embedding_matrix[i] = fasttext[word]
            found += 1
        else:
            missing.append(word)

print(f"\n✅ Palabras encontradas en FastText: {found}/{len(word2idx_inputs)}")
print(f"❌ Palabras faltantes (primeras 10): {missing[:10]}")
print(f"📐 Dimensiones del embedding_matrix: {embedding_matrix.shape}")

⏬ Cargando FastText...
🔄 Generando embedding matrix...

✅ Palabras encontradas en FastText: 3736/4046
❌ Palabras faltantes (primeras 10): ['convai', 'whazzup', 'buongiorno', '_', 'poyou', 'zitah', 'orhun', 'wontice', 'hesnt', 'robotcop']
📐 Dimensiones del embedding_matrix: (6000, 300)


### 🚩 4. Entrenar el modelo Encoder-Decoder (Seq2Seq)

In [192]:
# Parámetros
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_input_len,))
encoder_embedding = Embedding(MAX_VOCAB_SIZE, embedding_dim,
                              weights=[embedding_matrix],
                              input_length=max_input_len,
                              trainable=False)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_out_len,))
decoder_embedding_layer = Embedding(num_words_output, latent_dim)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Atención (Luong: dot product entre encoder_outputs y decoder_outputs)
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])         # (batch, dec_seq, enc_seq)
attention = Activation('softmax')(attention)                             # softmax sobre el encoder sequence
context = Dot(axes=[2,1])([attention, encoder_outputs])                 # contexto ponderado
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output final
output = Dense(256, activation='tanh')(decoder_combined_context)
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(output)

# Modelo final con atención
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/Users/lucianoadassus/Library/Caches/pypoetry/virtualenvs/intro-ia-MJmdkF7C-py3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_32"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_82      │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_83      │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, 25, 300)   │  1,800,000 │ input_layer_82[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, 25, 256)   │  1,067,776 │ input_layer_83[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_18 (LSTM)      │ [(None, 25, 256), │    570,368 │ embedding_22[0][… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ [(None, 25, 256), │    525,312 │ embedding_23[0][… │
│                     │ (None, 256),      │            │ lstm_18[0][1],    │
│                     │ (None, 256)]      │            │ lstm_18[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_38 (Dot)        │ (None, 25, 25)    │          0 │ lstm_19[0][0],    │
│                     │                   │            │ lstm_18[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 25, 25)    │          0 │ dot_38[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_39 (Dot)        │ (None, 25, 256)   │          0 │ activation_19[0]… │
│                     │                   │            │ lstm_18[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_19      │ (None, 25, 512)   │          0 │ dot_39[0][0],     │
│ (Concatenate)       │                   │            │ lstm_19[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 25, 256)   │    131,328 │ concatenate_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 25, 4171)  │  1,071,947 │ dense_23[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,166,731 (19.71 MB)

 Trainable params: 3,366,731 (12.84 MB)

 Non-trainable params: 1,800,000 (6.87 MB)

In [193]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callback para detener entrenamiento si el val_loss no mejora
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Callback para guardar el mejor modelo en archivo
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Entrenamiento
model.fit(
    [encoder_input_sequences, decoder_output_sequences],
    decoder_targets,
    batch_size=64,
    epochs=30,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 99s 581ms/step - accuracy: 0.6459 - loss: 3.1466 - val_accuracy: 0.7302 - val_loss: 1.7428
Epoch 2/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 88s 549ms/step - accuracy: 0.7341 - loss: 1.6877 - val_accuracy: 0.7513 - val_loss: 1.5822
Epoch 3/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 95s 599ms/step - accuracy: 0.7523 - loss: 1.5176 - val_accuracy: 0.7661 - val_loss: 1.4559
Epoch 4/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 92s 580ms/step - accuracy: 0.7709 - loss: 1.3509 - val_accuracy: 0.7733 - val_loss: 1.3955
Epoch 5/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 103s 646ms/step - accuracy: 0.7805 - loss: 1.2645 - val_accuracy: 0.7779 - val_loss: 1.3416
Epoch 6/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 101s 635ms/step - accuracy: 0.7911 - loss: 1.1656 - val_accuracy: 0.7820 - val_loss: 1.3027
Epoch 7/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 107s 669ms/step - accuracy: 0.7956 - loss: 1.1100 - val_accuracy: 0.7820 - val_loss: 1.2836
Epoch 8/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 107s 669ms/step - accuracy: 0.7969 - los

### 🚩 5. Inferencia

#### Encoder model para inferencia:

In [194]:
# Encoder model: devuelve encoder_outputs y estados
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

#### Decoder model reutilizando embeddings y atención:

In [195]:
# Inputs del decoder para inferencia
decoder_input_single = Input(shape=(1,))
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
encoder_outputs_input = Input(shape=(max_input_len, latent_dim))

decoder_embedding_inf = decoder_embedding_layer(decoder_input_single)

# LSTM paso a paso
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_inf,
    initial_state=[decoder_state_input_h, decoder_state_input_c]
)

# Atención
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs_input])
attention = Activation('softmax')(attention)
context = Dot(axes=[2, 1])([attention, encoder_outputs_input])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Capa final
output = Dense(256, activation='tanh')(decoder_combined_context)
decoder_outputs = decoder_dense(output)

# Modelo final de inferencia
decoder_model = Model(
    [decoder_input_single, decoder_state_input_h, decoder_state_input_c, encoder_outputs_input],
    [decoder_outputs, state_h, state_c]
)

#### Diccionario inverso para decodificar índices:

In [196]:
reverse_word2idx_outputs = {idx: word for word, idx in word2idx_outputs.items()}

#### Función decode_sequence() :

In [206]:
from collections import Counter

def decode_sequence_beam_super(input_seq, beam_width=3, max_repeat=3, min_prob=1e-6, length_penalty_alpha=0.6):
    enc_outs, h, c = encoder_model.predict(input_seq, verbose=0)
    sequences = [([word2idx_outputs['<sos>']], 0.0, h, c)]

    for _ in range(max_out_len):
        all_candidates = []

        for seq, score, h, c in sequences:
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = seq[-1]

            output_tokens, h_new, c_new = decoder_model.predict([target_seq, h, c, enc_outs], verbose=0)
            output_probs = output_tokens[0, -1, :]

            top_indices = output_probs.argsort()[-beam_width:][::-1]

            for idx in top_indices:
                if idx not in reverse_word2idx_outputs:
                    continue
                word = reverse_word2idx_outputs[idx]
                prob = output_probs[idx]

                if prob < min_prob:
                    continue

                new_seq = seq + [idx]
                if word == '<eos>':
                    decoded = [reverse_word2idx_outputs.get(i, "<UNK>") for i in new_seq[1:-1]]
                    return ' '.join(decoded)

                # Penalización por log(prob) y por longitud (más larga = mejor)
                length_penalty = ((5 + len(new_seq)) / 6) ** length_penalty_alpha
                candidate_score = (score - np.log(prob + 1e-10)) / length_penalty

                all_candidates.append((new_seq, candidate_score, h_new, c_new))

        if not all_candidates:
            break

        sequences = sorted(all_candidates, key=lambda tup: tup[1])[:beam_width]

    # Sin <eos> → devolver mejor secuencia
    final_sequence = sequences[0][0][1:]  # quitar <sos>
    decoded_words = [reverse_word2idx_outputs.get(i, "<UNK>") for i in final_sequence]

    # Evitar repeticiones
    word_counts = Counter(decoded_words)
    most_common_word, count = word_counts.most_common(1)[0]
    if count > max_repeat:
        decoded_words = list(dict.fromkeys(decoded_words))  # eliminar duplicados preservando orden

    # Evitar triples consecutivos
    cleaned = []
    for w in decoded_words:
        if len(cleaned) < 2 or not (w == cleaned[-1] == cleaned[-2]):
            cleaned.append(w)

    return ' '.join(cleaned)

#### Función para consultar al bot:

In [208]:
def answer_question_beam_super(question):
    question_clean = clean_text(question)
    seq = tokenizer_inputs.texts_to_sequences([question_clean])
    pad_seq = pad_sequences(seq, maxlen=max_input_len, padding='post')
    return decode_sequence_beam_super(pad_seq)

# Probar
test_questions = [
    "Do you prefer Android or iOS?",
    "Why did you choose to become a vegan?",
    "How do you stay healthy while living on donations?"
]

for q in test_questions:
    print(f"🧍 Question: {q}")
    print(f"🤖 Answer: {answer_question_beam_super(q)}\n")

🧍 Question: Do you prefer Android or iOS?
🤖 Answer: clubs lifting biggest royce ride sheep couch assessor christ reproduce served secondhand newspaper reset

🧍 Question: Why did you choose to become a vegan?
🤖 Answer: kills lifting draws ride everybodys polyglot teachers brownies served rad station fyi

🧍 Question: How do you stay healthy while living on donations?
🤖 Answer: competes lifting favorite ride mum em salsa friday wednesday softball reset



### 🚩 Resultados del Entrenamiento - Modelo de Chatbot con Atención

**✅ Estado actual del modelo:**

El modelo fue entrenado con las siguientes configuraciones:

- **Corpus reducido:** 12.648 pares pregunta-respuesta.
- **Vocabulario máximo:** 6.000 palabras.
- **Limpieza avanzada:** expansión de contracciones, remoción de emojis, puntuación, HTML, palabras repetidas y respuestas poco informativas.
- **Modelo con atención Luong y decodificación Beam Search penalizada por longitud.**

---

#### 📊 Comportamiento observado

Las respuestas generadas muestran:

- 🌟 **Mayor coherencia estructural**: ya no repite tokens como `"you you"` ni genera palabras sin sentido.
- 🧩 **Patrones sintácticos más variados**, aunque persisten ciertas "muletillas" como `"clubs lifting ride sheep..."` en varias respuestas.
- 🔁 **Tendencia a repetir construcciones**: ciertas frases o combinaciones aparecen frecuentemente, señal de que el modelo internalizó estructuras comunes del dataset.

---

#### 📉 Limitaciones detectadas

- Algunas respuestas **pierden el foco semántico** frente a preguntas abiertas.
- Persisten **muletillas repetidas** que indican sobreajuste a patrones comunes.
- El modelo **no capta completamente el sentido de la pregunta**, y responde con frases sueltas o listas de sustantivos.

---

#### 🔧 Siguientes pasos recomendados

1. **🔁 Mayor diversidad en el dataset:**
   - Aumentar la variedad semántica de las respuestas.
   - Eliminar frases tipo muletilla.
   - Incluir ejemplos con estructuras más variadas, especialmente preguntas-respuestas informativas.

2. **🧼 Limpieza semántica adicional:**
   - Eliminar repeticiones como `"i am a college student and i am a college student"`.
   - Filtrar respuestas con más de 2 repeticiones de la misma palabra.

3. **📈 Ajustes en la inferencia:**
   - Explorar `top-k` o `top-p` sampling para respuestas más diversas.
   - Usar `beam_width=5` y penalización de longitud más fuerte para evitar listas sin sentido.

4. **🔡 Mejora del embedding:**
   - Usar embeddings entrenables (`trainable=True`) para que el modelo pueda adaptarse mejor.
   - Considerar embeddings específicos del dominio si el tema lo permite.

---

#### 🏁 Conclusión

El modelo muestra una **mejora importante respecto a versiones anteriores**, especialmente gracias al preprocesamiento y beam search penalizado. Todavía puede mejorar en **coherencia semántica y riqueza léxica**, algo que se puede abordar en futuros entrenamientos con corpus más balanceado y estrategias de decodificación más exploratorias.